In [ ]:
using Pkg
Pkg.activate(".")
Pkg.develop(path="../../Tambo")

In [ ]:
using Tambo

In [ ]:
using Arrow
using Makie
using StaticArrays
using GLMakie
using LinearAlgebra

In [ ]:
new_events = Arrow.Table("/Users/jlazar/Downloads/injection_new_test.arrow").injected_events
old_events = Arrow.Table("/Users/jlazar/Downloads/injection_old_test.arrow").injected_events

In [ ]:
function closest_approach(event)
    h = norm(event.final_state.position)
    θ = acos(dot(event.final_state.position, event.final_state.direction.proj) / norm(event.final_state.position))
    d = h * cos(θ)
    return event.final_state.position .- d .* event.final_state.direction.proj
end

In [ ]:
sim = Simulation("../../resources/configuration_examples/larger_valley.toml")
geo = Tambo.Geometry(sim.config["geometry"])

In [ ]:
for (idx, event) in enumerate(new_events)
    closest = closest_approach(event)
    
    x_boundary = intersect(closest, reverse(event.final_state.direction), geo.box)

    track = Tambo.Track(closest, x_boundary)
    injection_position, no_endcap = Tambo.determine_injection_start(closest, x_boundary, geo)

    ps = []
    λs = LinRange(-0.5, 1, 100)

    for λ in λs
        p = track(λ)
        push!(ps, p)
    end

    xs = LinRange(-5, 10, 100) .* units.km
    ys = LinRange(-10, 10, 100) .* units.km
    zs = [geo(x, y) for x in xs, y in ys]

    fig = Figure()
    ax = Axis3(
        fig[1, 1],
        xlabel="x [km]",
        ylabel="y [km]",
        zlabel="z [km]",
        azimuth = -pi/2 + 0.2
    )

    xlims!(ax, -5, 10)
    ylims!(ax, -10, 10)
    zlims!(ax, -2, 3)
    
    surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=0.3)
    lines!(ax, ps ./ units.km, color=λs, colormap=:rainbow)

    color = dot(closest, geo.plane.n̂.proj) < 0 ? :red : :green
    strokecolor = Tambo.inside(closest, geo) ? :white : :black
    sca1 = scatter!(
        ax, [closest ./ units.km],
        overdraw=true,
        markersize=15,
        color=color,
        strokecolor=strokecolor,
        strokewidth=1
    )
    
    color = dot(injection_position, geo.plane.n̂.proj) <= 0 ? :red : :green
    strokecolor = Tambo.inside(injection_position, geo) ? :white : :black
    sca2 = scatter!(
        ax,
        [injection_position ./ units.km],
        overdraw=true,
        color=color,
        strokecolor=strokecolor,
        strokewidth=1
    )
    sca_tam = scatter!(
        ax,
        [SVector{3}([0, 0, 0])./ units.km],
        marker=:star5
    )
    
    leg = Legend(
        fig[1, 2],
        [sca1, sca2, sca_tam],
        ["Old injection", "New injection", "Tambo"]
    )

    save("compare_start/compare_start_$(idx).png", fig)
    
end

In [ ]:
for (idx, (old_event, new_event)) in enumerate(zip(old_events, new_events))
    
    closest = closest_approach(old_event)
    
    x_boundary = intersect(closest, reverse(old_event.final_state.direction), geo.box)

    track = Tambo.Track(closest, x_boundary)

    ps = []
    λs = LinRange(-0.5, 1, 100)

    for λ in λs
        p = track(λ)
        push!(ps, p)
    end
    
    xs = LinRange(-5, 10, 100) .* units.km
    ys = LinRange(-10, 10, 100) .* units.km
    zs = [geo(x, y) for x in xs, y in ys]

    fig = Figure()
    ax = Axis3(
        fig[1, 1],
        xlabel="x [km]",
        ylabel="y [km]",
        zlabel="z [km]",
        azimuth = -pi/2 + 0.2
    )

    xlims!(ax, -5, 10)
    ylims!(ax, -10, 10)
    zlims!(ax, -2, 3)
    
    surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=0.3)
    lines!(ax, ps ./ units.km, color=λs, colormap=:rainbow)

    color = dot(old_event.final_state.position, geo.plane.n̂.proj) < 0 ? :red : :green
    strokecolor = Tambo.inside(old_event.final_state.position, geo) ? :white : :black
    sca1 = scatter!(
        ax, [old_event.final_state.position ./ units.km],
        overdraw=true,
        markersize=15,
        color=color,
        strokecolor=strokecolor,
        strokewidth=1
    )
    
    color = dot(new_event.final_state.position, geo.plane.n̂.proj) < 0 ? :red : :green
    strokecolor = Tambo.inside(new_event.final_state.position, geo) ? :white : :black
    sca2 = scatter!(
        ax,
        [new_event.final_state.position ./ units.km],
        overdraw=true,
        color=color,
        strokecolor=strokecolor,
        strokewidth=1
    )
    sca_tam = scatter!(
        ax,
        [SVector{3}([0, 0, 0])./ units.km],
        marker=:star5
    )
    
    leg = Legend(
        fig[1, 2],
        [sca1, sca2, sca_tam],
        ["Old injection", "New injection", "Tambo"]
    )
    save("./compare_cc_vertex/compare_cc_vertex_$(idx).png", fig)
    
end

In [ ]:
xs = LinRange(-5, 10, 100) .* units.km
ys = LinRange(-10, 10, 100) .* units.km
zs = [geo(x, y) for x in xs, y in ys]

fig = Figure()
ax = Axis3(
    fig[1, 1],
    xlabel="x [km]",
    ylabel="y [km]",
    zlabel="z [km]",
    azimuth = -pi/2 + 0.2
)

plane_zs = [Tambo.plane_z(x, y, geo.plane) for x in xs, y in ys]

surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=1.0, colormap=:darkterrain)

surface!(ax, xs ./ units.km, ys ./ units.km, plane_zs ./ units.km, alpha=0.4)


xlims!(ax, -5, 10)
ylims!(ax, -10, 10)
zlims!(ax, -2, 3)

save("visualize_plane.png", fig)

In [ ]:
# function make_disk_surface(r::Number, d::Tambo.Direction)
    
#     ϕs = LinRange(0, 2π, 30)
#     rs = LinRange(floatmin(), r, 20)
#     ps = Point3[]
#     for ϕ in ϕs
#         for r in rs
#             push!(ps, [r * cos(ϕ), r * sin(ϕ), 0])
#         end
#     end
#     rot = RotZ(d.ϕ) * RotY(d.θ)
#     return [rot * p for p in ps]
# end

In [ ]:
# xs = LinRange(-5, 10, 100) .* units.km
# ys = LinRange(-10, 10, 100) .* units.km
# zs = [geo(x, y) for x in xs, y in ys]

# fig = Figure()
# ax = Axis3(
#     fig[1, 1],
#     xlabel="x [km]",
#     ylabel="y [km]",
#     zlabel="z [km]",
#     azimuth = -pi/2 + 0.2
# )

# surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=1.0, colormap=:darkterrain)

# xlims!(ax, -5, 10)
# ylims!(ax, -10, 10)
# zlims!(ax, -2, 3)


# a = make_disk_surface(2, geo.plane.n̂.proj)

# scatter!(ax, [SVector{3}([0,0,0])], marker=:star5, color=:red, markersize=20)
# scatter!(ax, map(x->x[1], a), map(x->x[2], a), map(x->x[3], a))

# save("visualize_mountain.png", fig)

In [ ]:
# xs = LinRange(-5, 10, 100) .* units.km
# ys = LinRange(-10, 10, 100) .* units.km
# zs = [geo(x, y) for x in xs, y in ys]

# for (idx, event) in enumerate(new_events[1:20])

#     closest = closest_approach(event)
    
#     x_boundary = intersect(closest, reverse(event.final_state.direction), geo.box)

#     track = Tambo.Track(closest, x_boundary)
#     injection_position, no_endcap = Tambo.determine_injection_start(closest, x_boundary, geo)

#     ps = []
#     λs = LinRange(-0.5, 1, 100)

#     for λ in λs
#         p = track(λ)
#         push!(ps, p)
#     end

#     xs = LinRange(-5, 10, 100) .* units.km
#     ys = LinRange(-10, 10, 100) .* units.km
#     zs = [geo(x, y) for x in xs, y in ys]

#     fig = Figure()
#     ax = Axis3(
#         fig[1, 1],
#         xlabel="x [km]",
#         ylabel="y [km]",
#         zlabel="z [km]",
#         azimuth = -pi/2 + 0.2
#     )

#     xlims!(ax, -5, 10)
#     ylims!(ax, -10, 10)
#     zlims!(ax, -2, 3)
    
#     surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=0.3, colormap=:darkterrain)
#     lines!(ax, ps ./ units.km, color=λs, colormap=:rainbow)

#     color = dot(closest, plane.n̂.proj) < 0 ? :red : :green
#     strokecolor = Tambo.inside(closest, geo) ? :white : :black
#     sca1 = scatter!(
#         ax, [closest ./ units.km],
#         overdraw=true,
#         markersize=15,
#         color=color,
#         strokecolor=strokecolor,
#         strokewidth=1
#     )
    
#     color = dot(injection_position, plane.n̂.proj) < 0 ? :red : :green
#     strokecolor = Tambo.inside(injection_position, geo) ? :white : :black
# #     sca2 = scatter!(
# #         ax,
# #         [injection_position ./ units.km],
# #         overdraw=true,
# #         color=color,
# #         strokecolor=strokecolor,
# #         strokewidth=1
# #     )
#     sca_tam = scatter!(
#         ax,
#         [SVector{3}([0, 0, 0])./ units.km],
#         marker=:star5
#     )
    
#     leg = Legend(
#         fig[1, 2],
#         [sca1],
#         ["Closest approach"]
#     )


#     a = make_disk_surface(2, event.final_state.direction.proj)
#     scatter!(ax, map(x->x[1], a), map(x->x[2], a), map(x->x[3], a))

#     save("visualize_disk/visualize_disk_$(idx).png", fig)
# end

In [ ]:
# n̂ = Tambo.Direction(geo.plane.n̂.θ, geo.plane.n̂.ϕ + Δϕ)
# plane = Tambo.Plane(n̂, geo.plane.x0)
# plane_zs = [Tambo.plane_z(x, y, plane) for x in xs, y in ys]

# fig = Figure()
# ax = Axis3(
#     fig[1, 1],
#     xlabel="x [km]",
#     ylabel="y [km]",
#     zlabel="z [km]",
#     azimuth = -pi/2 + 0.2
# )

# xlims!(ax, -5, 10)
# ylims!(ax, -10, 10)
# zlims!(ax, -2, 3)

# surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=1.0, colormap=:darkterrain)
# surface!(ax, xs ./ units.km, ys ./ units.km, plane_zs ./ units.km, alpha=0.4)

# scatter!(ax, [SVector{3}([0, 0, 0])])

# save("fadsfdsf.png", fig)

In [ ]:
# xs = LinRange(-5, 10, 100) .* units.km
# ys = LinRange(-10, 10, 100) .* units.km
# zs = [geo(x, y) for x in xs, y in ys]

# for (idx, Δϕ) in enumerate(LinRange(-0.3, 0.1, 20))
#     @show rad2deg(Δϕ)
        
#     n̂ = Tambo.Direction(geo.plane.n̂.θ, geo.plane.n̂.ϕ + Δϕ)
#     plane = Tambo.Plane(n̂, geo.plane.x0)
#     plane_zs = [Tambo.plane_z(x, y, plane) for x in xs, y in ys]

#     fig = Figure()
#     ax = Axis3(
#         fig[1, 1],
#         xlabel="x [km]",
#         ylabel="y [km]",
#         zlabel="z [km]",
#         azimuth = -pi/2 + 0.2
#     )

#     xlims!(ax, -5, 10)
#     ylims!(ax, -10, 10)
#     zlims!(ax, -2, 3)
    
#     surface!(ax, xs ./ units.km, ys ./ units.km, zs ./ units.km, alpha=1.0, colormap=:darkterrain)
#     surface!(ax, xs ./ units.km, ys ./ units.km, plane_zs ./ units.km, alpha=0.4)

#     scatter!(ax, [SVector{3}([0, 0, 0])])
    
# #     save("visualize_plane_$(idx).png", fig)
# end